In [1]:
import os
import sys

data_path = 'kth/'
motion_types = list(filter(lambda s: s.endswith('ing'), os.listdir(data_path)))
video_files = {}
for m in motion_types:
    video_files[m] = list(filter(lambda s: s.endswith('.avi'), os.listdir(data_path + m + '/')))
    video_files[m].sort()

In [2]:
import imageio
import numpy as np
import cv2
from PIL import Image

def read_video(path):
    reader = imageio.get_reader(path)

    fps = reader.get_meta_data()['fps']
    driving_video = []
    try:
        for im in reader:
            driving_video.append(im)
    except RuntimeError:
        pass
    reader.close()
    
    return fps, driving_video

In [6]:
from tqdm import tqdm

#train_dir = data_path + '128x128/train/'
test_dir = data_path + '128x128/test2/'

try:
    #os.mkdir(data_path + '128x128')
    #os.mkdir(train_dir)
    os.mkdir(test_dir)
except:
    pass

for m in motion_types:
    for f in tqdm(video_files[m]):
        # Read video
        file_name = data_path + m + '/' + f
        fps, video = read_video(file_name)
        person = int(f[6:8])
        #videos = breakdown(video)
        
        if person < 24:
            continue
        elif person == 24:
            continue
        else:
            new_file_name = test_dir + f[:-4]
        
        #for i, video in enumerate(videos):
        new_video = []
        for frame in video:
            frame = frame[:, 20:140]
            frame = cv2.resize(frame, dsize=(128, 128))
            frame = np.expand_dims(frame, 0)
            new_video.append(frame)
        new_video = np.concatenate(new_video, axis=0)
        imageio.mimsave(new_file_name + str(i).zfill(2) + '.mp4', new_video, fps=fps)

100%|██████████| 100/100 [00:09<00:00, 10.38it/s]


In [3]:
def breakdown(video):
    N = len(video) // 16
    videos = []
    for i in range(N):
        videos.append(video[i*16:(i+1)*16])
    return videos